# Original code from the class
- Divides the data into a training set (last quarter of 2018) and a test set (first day of 2019) based on date ranges.

In [ ]:
# import pandas as pd
# import numpy as np

# df = pd.read_csv("Bitstamp_BTCUSD_2018_minute.csv", header=1)
# df.index = df.date
# df.drop(["unix", "date", "symbol", "Volume USD"], axis=1, inplace=True)
# df = df.iloc[::-1] # Reverse the order of dates

# BTC_vol = df["Volume BTC"].values
# df_diff = df.diff()
# df_diff["Volume BTC"] = np.log(1 + BTC_vol)

# # Select the last quarter as the training dataset
# date_index = pd.to_datetime(df_diff.index)
# mask_train = pd.Series(date_index).between("2018-10-01", "2018-12-31",
#     inclusive="left")
# df_train = df_diff.loc[mask_train.values].copy(deep=True)
# y_close_train = df.close.loc[mask_train.values]
# train_close = df.loc[mask_train.values].close.values
# df_train["Relative_Close"] = train_close / train_close[0]

# # Select the first day as the test dataset
# mask_test = pd.Series(date_index).between("2018-12-31", "2019-01-01",
#     inclusive="left")
# df_test = df_diff.loc[mask_test.values].copy(deep=True)
# test_close = df.loc[mask_test.values].close.values
# df_test["Relative_Close"] = test_close / train_close[0]
# y_close_test = df.close[mask_test.values]

# print(df_train.columns.values)

# def generate_dataset(df, seq_len):
#     X_list, y_list = [], []
#     for i in range(len(df.index) - seq_len):
#         X_list.append(np.array(df.iloc[i:i+seq_len,:]))
#         y_list.append(df.close.values[i+seq_len])
#     return np.array(X_list), np.array(y_list)

In [ ]:
# LAG = 5
# # Add LAG number of observations in training dataset to test dataset
# df_test_LAG = pd.concat((df_train.iloc[-LAG:,:], df_test), axis=0)

# X_train, y_train = generate_dataset(df_train, seq_len=LAG)
# X_test, y_test = generate_dataset(df_test_LAG, seq_len=LAG)

# print(np.mean(y_train))
# print(np.mean((y_train - np.mean(y_train))**2))
# print(np.mean((y_test - np.mean(y_train))**2))

# import tensorflow as tf
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers import Adam

# tf.keras.utils.set_random_seed(4002)

# model = Sequential()
# model.add(LSTM(8, return_sequences=True))
# model.add(LSTM(8, dropout=0.2))

# model.add(Dense(1))
# model.@@compile@@(optimizer=Adam(learning_rate=0.001), loss='mse')
# model.fit(X_train, y_train, batch_size=64, epochs=30, shuffle=True)

# LSTM_pred = np.squeeze(model.predict(X_test))
# print(np.mean((LSTM_pred - y_test)**2))

## Changed to daily basis

In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [44]:
df = pd.read_csv('/content/2018-2019_bit.csv')

In [45]:
df = df.iloc[:,1:]

In [46]:
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2018-09-25 00:01:00,6598.88,6601.91,6598.88,6601.75,17.507869,115553.481670,6600.088175
1,2018-09-25 00:02:00,6600.02,6601.17,6600.00,6601.17,0.625429,4128.483922,6601.038358
2,2018-09-25 00:03:00,6601.89,6602.01,6601.32,6602.00,6.713058,44318.766584,6601.874691
3,2018-09-25 00:04:00,6602.65,6604.96,6602.00,6602.00,5.025637,33180.213169,6602.190509
4,2018-09-25 00:05:00,6607.51,6607.82,6606.24,6606.88,2.505021,16551.781741,6607.443549


In [47]:
df.columns = ['date', 'open', 'high', 'low', 'close', 'vol(btc)', 'vol', 'weighted_price']

In [48]:
df['date'] = pd.to_datetime(df['date'])

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142559 entries, 0 to 142558
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            142559 non-null  datetime64[ns]
 1   open            142559 non-null  float64       
 2   high            142559 non-null  float64       
 3   low             142559 non-null  float64       
 4   close           142559 non-null  float64       
 5   vol(btc)        142559 non-null  float64       
 6   vol             142559 non-null  float64       
 7   weighted_price  142559 non-null  float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 8.7 MB


In [50]:
df.index = pd.to_datetime(df['date'])

In [51]:
df.head()

,date,open,high,low,close,vol(btc),vol,weighted_price
date,,,,,,,,
2018-09-25 00:01:00,2018-09-25 00:01:00,6598.88,6601.91,6598.88,6601.75,17.507869,115553.481670,6600.088175
2018-09-25 00:02:00,2018-09-25 00:02:00,6600.02,6601.17,6600.00,6601.17,0.625429,4128.483922,6601.038358
2018-09-25 00:03:00,2018-09-25 00:03:00,6601.89,6602.01,6601.32,6602.00,6.713058,44318.766584,6601.874691
2018-09-25 00:04:00,2018-09-25 00:04:00,6602.65,6604.96,6602.00,6602.00,5.025637,33180.213169,6602.190509
2018-09-25 00:05:00,2018-09-25 00:05:00,6607.51,6607.82,6606.24,6606.88,2.505021,16551.781741,6607.443549


In [52]:
df.drop(["date", "vol(btc)", 'weighted_price'], axis=1, inplace=True)

In [53]:
# Prepare the volume and price differences, normalize volume
BTC_vol = df["vol"].values
df_diff = df.diff().dropna()
df_diff["vol"] = np.log(1 + BTC_vol[:-1])  # Shifted by 1 to align with diff

In [54]:
df_aligned = df.loc[df_diff.index]

In [55]:
mask_train = (df_diff.index >= "2018-10-01") & (df_diff.index < "2018-12-31")

In [56]:
df_train = df_diff.loc[mask_train].copy()

In [57]:
df_train

,open,high,low,close,vol
date,,,,,
2018-10-01 00:00:00,-3.05,-1.58,0.05,0.05,9.311424
2018-10-01 00:01:00,-0.52,-1.99,-2.89,-2.89,9.206623
2018-10-01 00:02:00,-4.96,-4.96,-6.67,-3.92,10.174242
2018-10-01 00:03:00,-3.30,-3.30,0.04,-0.02,9.247930
2018-10-01 00:04:00,0.04,1.55,2.73,1.13,8.408290
...,...,...,...,...,...
2018-12-30 23:55:00,-1.25,-1.25,-4.57,-3.89,6.418919
2018-12-30 23:56:00,-6.00,-6.00,-2.68,-3.36,9.618290
2018-12-30 23:57:00,2.25,2.25,2.25,2.25,5.989581


In [58]:
# Assuming df_diff has been created by df.diff().dropna() or similar
# First, align the index of df to match df_diff after dropping NaN values

# Now, proceed with mask creation and selection
df_train = df_diff.loc[mask_train].copy()

# Use df_aligned to ensure the indexes match
train_close = df_aligned.loc[mask_train, "close"].values

# Since indexes are aligned, this operation should now work without error
df_train["Relative_Close"] = train_close / train_close[0]

In [59]:
df_train.head()

,open,high,low,close,vol,Relative_Close
date,,,,,,
2018-10-01 00:00:00,-3.05,-1.58,0.05,0.05,9.311424,1.000000
2018-10-01 00:01:00,-0.52,-1.99,-2.89,-2.89,9.206623,0.999560
2018-10-01 00:02:00,-4.96,-4.96,-6.67,-3.92,10.174242,0.998963
2018-10-01 00:03:00,-3.30,-3.30,0.04,-0.02,9.247930,0.998960
2018-10-01 00:04:00,0.04,1.55,2.73,1.13,8.408290,0.999132


In [60]:
mask_test = (df_diff.index >= "2018-12-31") & (df_diff.index < "2019-01-01")  # December 2018 for testing
df_test = df_diff.loc[mask_test].copy()
test_close = df_aligned.loc[mask_test, "close"].values
df_test["Relative_Close"] = test_close / train_close[0]

In [61]:
df_test.head()

,open,high,low,close,vol,Relative_Close
date,,,,,,
2018-12-31 00:00:00,-1.94,3.06,-1.65,3.90,10.378692,0.579045
2018-12-31 00:01:00,3.43,1.50,5.08,2.60,10.368279,0.579440
2018-12-31 00:02:00,2.81,-0.26,2.81,-0.26,4.599903,0.579401
2018-12-31 00:03:00,-0.17,0.25,-1.23,-1.23,7.900077,0.579214
2018-12-31 00:04:00,-0.50,-0.92,-2.97,0.56,8.638780,0.579299


In [62]:
# Generate dataset function
def generate_dataset(df, seq_len):
    X_list, y_list = [], []
    for i in range(len(df) - seq_len):
        X_list.append(df.iloc[i:(i+seq_len), :].values)
        y_list.append(df["close"].iloc[i + seq_len])
    return np.array(X_list), np.array(y_list)

In [64]:
LAG = 5 # Example: Use the past # days to predict the next day

In [65]:
# Prepare training and test datasets
X_train, y_train = generate_dataset(df_train, LAG)
X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [66]:
print(np.mean(y_train))

-0.021087877284694932


In [67]:
print(np.mean((y_train-np.mean(y_train))**2))

25.5640603672649


In [68]:
np.mean(y_train)

-0.021087877284694932

In [69]:
print(np.mean((y_train - np.mean(y_train))**2))

25.5640603672649


In [70]:
# Model architecture (unchanged from the minute-by-minute example)
tf.keras.utils.set_random_seed(4002)
model = Sequential([
    LSTM(8, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(8, dropout=0.2),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [71]:
# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=30, shuffle=True)

Epoch 1/30
2048/2048 [==============================] - 20s 7ms/step - loss: 25.5469
Epoch 2/30
2048/2048 [==============================] - 15s 7ms/step - loss: 25.4614
Epoch 3/30
2048/2048 [==============================] - 15s 8ms/step - loss: 25.4150
Epoch 4/30
2048/2048 [==============================] - 14s 7ms/step - loss: 25.3746
Epoch 5/30
2048/2048 [==============================] - 15s 7ms/step - loss: 25.3568
Epoch 6/30
2048/2048 [==============================] - 15s 7ms/step - loss: 25.3478
Epoch 7/30
2048/2048 [==============================] - 14s 7ms/step - loss: 25.3532
Epoch 8/30
2048/2048 [==============================] - 15s 7ms/step - loss: 25.3207
Epoch 9/30
2048/2048 [==============================] - 15s 7ms/step - loss: 25.3219
Epoch 10/30
2048/2048 [==============================] - 15s 7ms/step - loss: 25.2946
Epoch 11/30
2048/2048 [==============================] - 16s 8ms/step - loss: 25.3048
Epoch 12/30
2048/2048 [==============================] - 15s 7m

In [72]:
# Predict and evaluate
LSTM_pred = np.squeeze(model.predict(X_test))
print("Test MSE:", np.mean((LSTM_pred - y_test)**2))

45/45 [==============================] - 1s 3ms/step
Test MSE: 16.646817618707686


In [73]:
LSTM_pred

array([ 0.6510125 , -0.4553829 , -0.2798961 , ..., -0.13670424,
       -0.00843152, -0.22609726], dtype=float32)

In [74]:
y_test

array([ 3.9 ,  2.6 , -0.26, ...,  2.22,  0.19,  1.44])